# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены. 
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок (insertion), удалений (deletion) и замен (substitution), соответственно. C - количество правильно распознанных слов (correct words).

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 20 баллов. 4 за сдачу в срок и 16 за задания: 
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов
* функция подсчета WER и ошибки пунктуации (тест 2.a) - 4 балла
* функция подсчета SA-WER (тест 3.а) - 4 балла

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER 


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [74]:
import re

def calculate_wer(reference_text: str, recognized_text: str) -> float:
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_words = re.sub(r'[^\w\s]', '', reference_text.lower()).split()
    recognized_words = re.sub(r'[^\w\s]', '', recognized_text.lower()).split()
    
    len_reference_words = len(reference_words)
    len_recognized_words = len(recognized_words)
    
    # Заполнение матрицы расстояний Левенштейна методом динамического программирования
    current_row = range(len_reference_words + 1)
    for i in range(1, len_recognized_words + 1):
        previous_row, current_row = current_row, [i] + [0] * len_reference_words
        for j in range(1, len_reference_words + 1):
            insertion, deletion = previous_row[j] + 1, current_row[j - 1] + 1
            substitution = previous_row[j - 1] + (0 if recognized_words[i - 1] == reference_words[j - 1] else 1)
            current_row[j] = min(insertion, deletion, substitution)
        
    # Расчет WER  (в процентах)
    wer = 100 * current_row[-1] / len_reference_words
    return wer

wer = calculate_wer('Я ел, солонину!', 'Я ел. Слона')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 33.33%


In [70]:
import re

def calculate_wer(reference_text: str, recognized_text: str) -> float:
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_words = re.sub(r'[^\w\s]', '', reference_text.lower()).split()
    recognized_words = re.sub(r'[^\w\s]', '', recognized_text.lower()).split()

    # расстояние Левенштейна 
    len_reference_words = len(reference_words)
    len_recognized_words = len(recognized_words)
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len_recognized_words + 1) for _ in range(len_reference_words + 1)]

    # Наполнение первой строки матрицы
    for i in range(len_reference_words + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len_recognized_words + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len_reference_words + 1):
        for j in range(1, len_recognized_words + 1):
            insertion, deletion = distance_matrix[i][j - 1] + 1, distance_matrix[i - 1][j] + 1
            substitution = distance_matrix[i - 1][j - 1] + (0 if reference_words[i - 1] == recognized_words[j - 1] else 1)
            distance_matrix[i][j] = min(insertion, deletion, substitution)

    # Расчет WER  (в процентах)
    wer = 100 * distance_matrix[-1][-1] / len_reference_words
    return wer

wer = calculate_wer('Я ел, солонину!', 'Я ел. Слона')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 33.33%


In [72]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"
    
def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")
    
test_wer() 
    

Test 1.a passed


## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания. 

Значение WER получается из трех видов ошибок: 
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется. 

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга. 

пример выравнивания: 

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [1]:
!pip install tabulate
from tabulate import tabulate
# используйте tabulate для отладки

In [4]:
import re

def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_words = re.sub(r'[^\w\s]', '', reference_text.lower()).split()
    recognized_words = re.sub(r'[^\w\s]', '', recognized_text.lower()).split()

    # расстояние Левенштейна 
    len_reference_words = len(reference_words)
    len_recognized_words = len(recognized_words)
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len_recognized_words + 1) for _ in range(len_reference_words + 1)]

    # Наполнение первой строки матрицы
    for i in range(len_reference_words + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len_recognized_words + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len_reference_words + 1):
        for j in range(1, len_recognized_words + 1):
            insertion, deletion = distance_matrix[i][j - 1] + 1, distance_matrix[i - 1][j] + 1
            substitution = distance_matrix[i - 1][j - 1] + (0 if reference_words[i - 1] == recognized_words[j - 1] else 1)
            distance_matrix[i][j] = min(insertion, deletion, substitution)

    # Инициализация словаря для хранения передаваемых переменных
    building_alignment =  {"wer": 0, # WER
                           "cor": 0, # инициализация счетчика для количества совпадений
                           "del": 0, # инициализация счетчика для количества удалений
                           "ins": 0, # инициализация счетчика для количества вставок
                           "sub": 0, # инициализация счетчика для количества замен
                           "ali": [[], [], []]} # инициализация пустых списков для хранения выровненных слов и типов операций
    # Расчет WER  (в процентах)
    building_alignment['wer'] = 100 * distance_matrix[-1][-1] / len_reference_words

    # Инициализация индексов для итерации по словам
    i, j = len_reference_words, len_recognized_words

    while i > 0 or j > 0:
        # Определение возможных шагов для текущей позиции в матрице Левенштейна
        x = distance_matrix[i][j - 1] if j > 0 else float('inf')
        y = distance_matrix[i - 1][j - 1] if i > 0 and j > 0 else float('inf')
        z = distance_matrix[i - 1][j] if i > 0 else float('inf')

        # Нахождение минимального шага
        step = min(x, y, z)

        # по диагонали (замена или совпадение)
        if step == y:
            if reference_words[i - 1] == recognized_words[j - 1]:
                # Совпадение
                building_alignment['ali'][2].append('C')
                building_alignment['cor'] += 1
            else:
                # Замена
                building_alignment['ali'][2].append('S')
                building_alignment['sub'] += 1
            building_alignment['ali'][0].append(reference_words[i - 1])
            building_alignment['ali'][1].append(recognized_words[j - 1])
            i -= 1
            j -= 1

        # по вертикали (удаление)
        elif step == z:
            building_alignment['ali'][0].append(reference_words[i - 1])
            building_alignment['ali'][1].append("***")
            building_alignment['ali'][2].append('D')
            building_alignment['del'] += 1
            i -= 1

        # по горизонтали (вставка)
        else:
            building_alignment['ali'][0].append("***")
            building_alignment['ali'][1].append(recognized_words[j - 1])
            building_alignment['ali'][2].append('I')
            building_alignment['ins'] += 1
            j -= 1

    # Разворачиваем созданные списки слов, т.к. собирали их с конца предложения
    building_alignment['ali'][0] = building_alignment['ali'][0][::-1] # разбитый по словам референс. Втавки отабражаются в эталонном выравнивании с помощью "***"
    building_alignment['ali'][1] = building_alignment['ali'][1][::-1] # разбитая по словам гипотеза.
    building_alignment['ali'][2] = building_alignment['ali'][2][::-1] # аннотация
    
    return building_alignment

In [1]:
def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
    return {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]}

In [5]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1, 
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1, 
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1, 
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})

    print(f"Test 1.b passed")
    
test_wer_with_alignment() 

Test 1.b passed


# 2. WER с пунктуацией (4 балла)
Попробуйте модифицировать WER таким образом, чтобы получившаяся метрика учитавала ошибки расстановки знаков препинания. 

Для этого надо ввести ограничение в алгоритм подсчета distance_matrix таким образом, чтобы запретить делать замену знака препинания на слово и наоборот.

Пример выравнивания 
```
Я сегодня  .   ***   ***  А ты  
Я    с    *** завтра  ?   А ты  
C    S    D_p   I    I_p  C  C    
```
Здесь суффикс _p в аннотации к ошибкам означает **ошибки пунктуации**


Задание: 
Напишите функцию, которая кроме стандартного WER считает дополнительно PunctuaionErrorRate (PER) по формуле

$$ PER = {I_p + D_p + S_p \over D_p + S_p + C_p} $$


In [ ]:
def calculate_wer_per(reference_text: str, recognized_text: str):
    # TODO   
    return {"wer" : wer,
            "per": per, 
            "ali": ali}


In [ ]:
def assert_wer_per(ref, hyp, ideal_report):
    report = calculate_wer_per(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_per():
    assert_wer_per('привет студент.', 'привет студент',  {
            "wer" : 0,
            "per": 100})
    assert_wer_per('привет студент.', 'студент.', {
            "wer" : 50,
            "per": 0,})
    assert_wer_per('привет студент.', 'привет. студент',  {
            "wer" : 0,
            "per": 200})
    assert_wer_per('привет студент.', '.студент?', {
            "wer" : 50,
            "per": 200, })

    print(f"Test 2 passed")
    
test_wer_per() 

# 3. Speaker-attributed Word Error Rate (4 балла)

В задаче распознавания диалоговых данных, когда говорят два диктора, важно не только распознать правильно каждое слово, но и отнести его к правильному диктору. Чаще всего такой результат получается с помощью комбинации двух независимых систем: распознавания речи и диаризация. 

При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect).

$$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$

Кроме подсчета самой ошибки, sa-wer решает еще одну задачку - поиск маппинга из эталонных названий спикеров (например, имен) в предсказанные (чаще всего Idшники). Это необходимо, тк система диаризации не знает, какие названия у спикеров в эталоне. При подсчете SA-WER проверяются все возможные мапинги спикеров и выбирается тот, который соответствует минимальному значению ошибки. 



In [118]:
from itertools import permutations, product

def calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers):
    # В отличие от прошлых функций на вход sawer подаются уже разбитые на слова произнесения
    # Кроме списка слов, дополнительно передается список меток спикеров
    assert isinstance(reference_text, list)
    assert isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)
    
    # TODO  посчитайте sawer с учетом мапинга спикеров
    # для этого посчитайте значение ошибки для каждого варианта мапинга меток дикторов 
    # и выберете тот, который соответствует минимальному SA-WER

    # Получаем уникальных дикторов из эталона и распознанного текста
    count_reference_speakers = list(set(reference_speakers))
    count_recognized_speakers = list(set(recognized_speakers))
    
    # Создаем все комбинации маппинга дикторов
    speaker_mapping_combinations = product(count_recognized_speakers, repeat = len(count_reference_speakers))

    best_sawer = float("inf")
    ali = {}
    len_reference_text = len(reference_text)
    len_recognized_text = len(recognized_text)

    for mapping in speaker_mapping_combinations:
        speaker_mapping = {count_reference_speakers[i]: mapping[i] for i in range(len(count_reference_speakers))}

        insertion, deletion, substitution, correct, wrong = 0, 0, 0, 0, 0 # Вставка, удаление, замена, правильный, неправильный диктор
        i, j = 0, 0
        
        while i < len_reference_text and j < len_recognized_text:

            if reference_text[i] == recognized_text[j]:
                if speaker_mapping.get(reference_speakers[i]) == recognized_speakers[j]:
                    correct += 1
                else:
                    wrong += 1
                i += 1
                j += 1
            else:
                if reference_text[i] != recognized_text[j]:
                    substitution += 1
                i += 1
                j += 1
        
        if i < len_reference_text:
            deletion += len_reference_text - i
            i = len_reference_text

        if j < len_recognized_text:
            insertion += len_recognized_text - j
            j = len_recognized_text

        # Расчет SA-WER
        total = deletion + substitution + correct + wrong
        sawer = (insertion + deletion + substitution + wrong) / total * 100 if total != 0 else 0
        # Выбираем минимальный SA-WER
        if sawer < best_sawer:
            best_sawer = sawer
            ali = speaker_mapping

    for i in count_recognized_speakers:
        if i not in ali.values():
            ali[None] = i

    return {"sawer": best_sawer, "ali": ali}

In [119]:
def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]

    
def test_sawer():
    assert_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 0})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 0})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1],  {
            "sawer" : 50})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [1, 0],  {
            "sawer" : 100})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 100})

    print(f"Test 3 passed")
    
test_sawer()

Test 3 passed
